In [28]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")
include("./trajopt/constraint.jl")
include("./funlopt/funl_dynamics.jl")
include("./funlopt/funl_utils.jl")
include("./funlopt/funl_synthesis.jl")
include("./funlopt/funl_constraint.jl")
include("./trajopt/scaling.jl")
using Plots
using Random

In [29]:
using MuJoCo
import MuJoCo: step! as mj_step
using Interpolations
using MatrixEquations

In [30]:
# load nominal trajectory
using JLD2, FileIO
@load "./data/funl_quad_star_0713" my_dict
x = my_dict["x"]
u = my_dict["u"]
t = my_dict["t"];
Q = my_dict["Q"];
Y = my_dict["Y"];
Z = my_dict["Z"];
alpha = my_dict["alpha"]
N = size(x,2) - 1
dt = zeros(N)
for i in 1:N
    dt[i] = t[i+1]-t[i]
end

In [42]:
dynamics = QuadrotorDynamics()
ix = dynamics.ix
iu = dynamics.iu
DLMI = LinearFOH(alpha,ix,iu)
# uf = [44.145;19.62;4.905]
r = 5.0
h = 5.0
MP_const = QuadrotorMultiphaseConstraint(15,r,h)
P1 = MP_const.P1
P2 = MP_const.P2
P3 = MP_const.P3
P4 = MP_const.P4
P5 = MP_const.P5

model = load_model("./xmls/quadrotor_star.xml")
data = init_data(model)

MuJoCo Data object

In [43]:
init_visualiser()
visualise!(model, data)

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


In [44]:
dt_control = model.opt.timestep
sim_time = 60.0
t_ctrl = range(0,sim_time,step=dt_control)

0.0:0.005:60.0

In [45]:
x_fit = [LinearInterpolation(t, x[idx,:],extrapolation_bc=Flat()) for idx in 1:ix]
u_fit = [LinearInterpolation(t, u[idx,:],extrapolation_bc=Flat()) for idx in 1:iu]
Q_fit = [ [LinearInterpolation(t, Q[i,j,:], extrapolation_bc=Flat()) for j in 1:ix] for i in 1:ix ]
Y_fit = [ [LinearInterpolation(t, Y[i,j,:], extrapolation_bc=Flat()) for j in 1:ix] for i in 1:iu ];
function get_x_interp(t::Float64)
    ans = zeros(ix)
    for i in 1:ix
        ans[i] = x_fit[i](t)
    end
    return ans
end
function get_u_interp(t::Float64)
    ans = zeros(iu)
    for i in 1:iu
        ans[i] = u_fit[i](t)
    end
    return ans
end
function get_Q_interp(t::Float64)
    new_Q_matrix = Array{Float64}(undef, ix, ix)
    for i in 1:ix
        for j in 1:ix
            new_Q_matrix[i, j] = Q_fit[i][j](t)
        end
    end
    return new_Q_matrix
end
function get_Y_interp(t::Float64)
    new_Y_matrix = Array{Float64}(undef, iu, ix)
    for i in 1:iu
        for j in 1:ix
            new_Y_matrix[i, j] = Y_fit[i][j](t)
        end
    end
    return new_Y_matrix
end

get_Y_interp (generic function with 1 method)

In [47]:
function euler_to_quaternion(attitude)
    roll, pitch, yaw = attitude[1],attitude[2],attitude[3]
    qx = sin(roll/2) * cos(pitch/2) * cos(yaw/2) - cos(roll/2) * sin(pitch/2) * sin(yaw/2)
    qy = cos(roll/2) * sin(pitch/2) * cos(yaw/2) + sin(roll/2) * cos(pitch/2) * sin(yaw/2)
    qz = cos(roll/2) * cos(pitch/2) * sin(yaw/2) - sin(roll/2) * sin(pitch/2) * cos(yaw/2)
    qw = cos(roll/2) * cos(pitch/2) * cos(yaw/2) + sin(roll/2) * sin(pitch/2) * sin(yaw/2)
    return [qw, qx, qy, qz]
end
function quaternion_to_euler(quat)
    w,x,y,z = quat[1],quat[2],quat[3],quat[4]
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    roll_x = atan(t0, t1)
    
    t2 = +2.0 * (w * y - z * x)
    t2 = t2 > 1.0 ? 1.0 : t2
    t2 = t2 < -1.0 ? -1.0 : t2
    pitch_y = asin(t2)
    
    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    yaw_z = atan(t3, t4)
    
    return [roll_x, pitch_y, yaw_z]
end
function mujocos_to_s(state)
    ans = zeros(ix)
    ans[1:3] .= state[1:3]
    ans[4:6] .= state[8:10]
    ans[7:9] .= quaternion_to_euler(state[4:7])
    ans[10:12] .= state[11:13]
    return ans
end
function s_to_mujocos(state)
    ans = zeros(ix+1)
    ans[1:3] .= state[1:3]
    ans[4:7] .= euler_to_quaternion(state[7:9])
    ans[8:10] .= state[4:6]
    ans[11:13] .= state[10:12]
    return ans
end
dx = 0.14
dy = 0.18
c = 0.0201
N = [1 1 1 1;-dy dy dy -dy;dx dx -dx -dx;-c c -c c]
N_inv = inv(N)
function actuator_allocation(u::Vector)
    return N_inv * u
end

actuator_allocation (generic function with 1 method)

In [50]:
xs_list = []
for i in 1:5
    z = randn(ix)
    z = z / norm(z)
    push!(xs_list,x[:,1] + sqrt(Q[:,:,1]) * z)
end

In [51]:
ctrl_trajectories = []
for (idx_s,xs) in enumerate(xs_list)
    reset!(model,data)
    A = mj_zeros(model.nq + model.nv + model.na)
    A[1,:] .= s_to_mujocos(xs)
    set_physics_state!(model,data,A')
    forward!(model,data)
    ctrl_states = zeros(13, length(t_ctrl))
    for (idx,t_) in enumerate(t_ctrl)
        @assert(isapprox(t_, data.time))
        ctrl_states[:,idx] = get_physics_state(model, data)
        ctrl_state_euler = mujocos_to_s(ctrl_states[:,idx])
        t_lap = t_ % t[end]

        Q_ = get_Q_interp(t_lap)
        Y_ = get_Y_interp(t_lap)
        K_ = Y_ * inv(Q_)
        x_ = get_x_interp(t_lap)
        u_ = get_u_interp(t_lap)

        u_ctrl = u_ + K_ * (ctrl_state_euler - x_)
        data.ctrl .= actuator_allocation(u_ctrl)
        mj_step(model, data)
    end
    push!(ctrl_trajectories,ctrl_states)
end

In [52]:
init_visualiser()
visualise!(model, data, trajectories = [ctrl_trajectories[i] for i in 1:length(xs_list)])

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


┌ Info: Recording video. Window resizing temporarily disabled
└ @ VisualiserExt /Users/taewankim/.julia/packages/MuJoCo/Ge60F/ext/VisualiserExt/functions.jl:186
┌ Info: Recording finished, window resizing re-enabled. Waiting for transcoding to finish.
└ @ VisualiserExt /Users/taewankim/.julia/packages/MuJoCo/Ge60F/ext/VisualiserExt/functions.jl:200
┌ Info: Finished recording! Video saved to /Users/taewankim/Documents/Code/Personal/Funnel_CTCS/jl_Le16HODApc.mp4
└ @ VisualiserExt /Users/taewankim/.julia/packages/MuJoCo/Ge60F/ext/VisualiserExt/functions.jl:204
┌ Info: Recording video. Window resizing temporarily disabled
└ @ VisualiserExt /Users/taewankim/.julia/packages/MuJoCo/Ge60F/ext/VisualiserExt/functions.jl:186
┌ Info: Recording finished, window resizing re-enabled. Waiting for transcoding to finish.
└ @ VisualiserExt /Users/taewankim/.julia/packages/MuJoCo/Ge60F/ext/VisualiserExt/functions.jl:200
┌ Info: Finished recording! Video saved to /Users/taewankim/Documents/Code/Personal/F

In [ ]:
reset!(model, data)

# Propagate derived quantities
mj_forward(model, data)
# Set joint accelerations to 0
data.qacc .= 0

# Inspect forces from inverse dynamics
mj_inverse(model, data)
println("Required control: ", data.qfrc_inverse)